In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [ ]:
path = r'D:\Documents\MSc DS\GDP Assignment'
df_gsdp_master=pd.read_csv(path+'/State-wise Gross Domestic Product (GDP) at current Price on yearly basis.csv')

In [ ]:
df_gsdp_growth=df_gsdp_master[df_gsdp_master['Items  Description'].str.contains('Growth')].reset_index()
df_gsdp_growth.drop(['index', 'Items  Description'], axis=1, inplace=True)

In [ ]:
UTs=['Delhi', 'Andaman & Nicobar Islands', 'Chandigarh']
all_nas=['West Bengal1']
df_gsdp_growth.drop(UTs, inplace=True, axis=1)
df_gsdp_growth.drop(all_nas, inplace=True, axis=1)
df_gsdp_growth=df_gsdp_growth[df_gsdp_growth.Duration != '2016-17']

In [ ]:
df_growth=df_gsdp_growth.melt(id_vars='Duration')
df_growth.columns=['Year', 'State', 'Growth']
df_growth['Growth'].fillna(method='ffill', inplace=True)

In [ ]:
df_growth['End Year'] = df_growth['Year'].apply(lambda x : x.split('-')[1]).astype(int)
df_growth['Growth']    = df_growth['Growth'].astype(float)
df_growth=df_growth.reset_index(drop=True)

In [ ]:
sns.lmplot(x='End Year', y='Growth', col='State', 
           col_wrap=8, height=3, truncate=True, 
           data=df_growth[df_growth.State != 'All_India GDP']).set(ylim=(-15, 25))
plt.show()

In [ ]:
sns.lmplot(x='End Year', y='Growth', col='State', 
           col_wrap=5, height=3, truncate=True, 
           legend_out = False,
           data=df_growth[df_growth.State == 'All_India GDP']).set(ylim=(-15, 25)).axes[0].set_title('All India')
    
plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 14
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size

In [ ]:
avg_growth = df_growth[df_growth.State != 'All_India GDP'].\
                    groupby('State')['Growth'].mean().reset_index().\
                    sort_values('Growth', ascending=False).reset_index().\
                    drop('index', axis=1)

In [ ]:
colours = ['cyan' if s != 'Uttar Pradesh' else 'green' for s in avg_growth.State]
colours[:3]  = ['blue']*3
colours[-3:] = ['gray']*3
sns.barplot(x='Growth', y='State', data=avg_growth, orient ='h', palette=colours)
plt.title('Averga GDP Growth Rate of Indian States', fontsize=15)
plt.show()    
###############################################################################

In [ ]:
india_growth = float (df_growth[df_growth.State == 'All_India GDP'].groupby('State')['Growth'].mean().reset_index()['Growth'])
print ('The 3 fastest growing states are: ', ', '.join(list(avg_growth.State.head(3))))
print ('The 3 slowest growing states are: ', ', '.join(list(avg_growth.State.tail(3))))
print ("India's average growth rate is", india_growth)
print ("My homestate Uttar Pradesh's growth rate is", float(avg_growth[avg_growth.State=='Uttar Pradesh'].Growth))

In [ ]:
df_gsdp_absolute=df_gsdp_master[df_gsdp_master.Duration=='2015-16']

In [ ]:
df_gsdp_absolute = df_gsdp_absolute[ list(set(df_gsdp_absolute) - \
                                {'Duration', 'Delhi', 'Chandigarh', 'Puducherry', 
                                 'Andaman & Nicobar Islands', 'Items  Description'})]

In [ ]:
df_gsdp_absolute = df_gsdp_absolute.transpose().reset_index()
df_gsdp_absolute.columns = ['State', 'GDP', 'Growth']

In [ ]:
df_gsdp_absolute = df_gsdp_absolute.dropna()
df_gsdp_absolute=df_gsdp_absolute.sort_values(by='GDP', ascending=False).reset_index(drop=True)

In [ ]:
pal         = sns.color_palette("Blues_d", len(df_gsdp_absolute))
growth_rank = df_gsdp_absolute.Growth.argsort().argsort()
df_gsdp_absolute=df_gsdp_absolute[df_gsdp_absolute.State != 'All_India GDP']
ax=sns.barplot(y='State', x='GDP', palette=np.array(pal[::-1])[growth_rank], data=df_gsdp_absolute)
i=1
for bar in ax.patches:
    x = bar.get_x() + bar.get_width() 
    y = bar.get_y() + bar.get_height()/2
    ax.annotate(str(df_gsdp_absolute.at[i, 'Growth'])+'%', (x, y))    
    i += 1
plt.title('GDP Of States along with their Growth Rate')
plt.show()

In [ ]:
print ("The Top 5 growing states for the financial year 2015-16 are:\n  ", 
       "\n   ".join(list(df_gsdp_absolute.head().State) ))

print ()
print ('The States which are doing poorly as they have both the low GDP growth and low GDP are Odisha, Meghalaya and Sikkim.')

In [ ]:
plt.rcParams["figure.figsize"] = [12, 8]

In [ ]:
UT_list      = ['Delhi', 'Chandigarh', 'Puducherry', 'Andaman & Nicobar Islands']
file_columns = ['SerialNum', 'Item', 'Amount']

In [ ]:
master_df = pd.DataFrame(columns=['State', 'Item', 'Amount', 'Category'])
for data_file in glob(path+'/NAD*.csv'):
    state = data_file.split('-')[1].replace('_', ' ')
    file_df = pd.read_csv(data_file, usecols=[0, 1, 5], encoding='cp1252')
    file_df.columns = file_columns
    file_df['Category'] = 'O'
    file_df['State']    = state
    file_df.iloc[0:6,   3] = 'P'
    file_df.iloc[7:10,  3] = 'S'
    file_df.iloc[11:26, 3] = 'T'
    file_df.loc[file_df.SerialNum == 'Total', ['Category']] = 'G'
    file_df.loc[file_df.SerialNum == 'Total', ['Item']] = file_df.loc[file_df.SerialNum == 'Total', ['Item']]+'Total'
    file_df = file_df[~(file_df.SerialNum.str.contains('\.'))]
    file_df.drop(['SerialNum'], inplace=True, axis=1)
    master_df = master_df.append(file_df)

In [ ]:
master_df=master_df[~(master_df.State.isin(UT_list))]
master_df.reset_index(drop=True, inplace=True)
master_df = master_df[['State', 'Item', 'Amount', 'Category']]
##########################################################################################################

In [ ]:
ppp_gsdp_df= master_df.loc[master_df.Item == 'Per Capita GSDP (Rs.)', ['State', 'Amount']].sort_values('Amount', ascending=False)
sns.barplot(y='State', x='Amount', data=ppp_gsdp_df, palette='Blues_d')
plt.title('Per Capita GDP of States of India', fontsize=14)
plt.show()

In [ ]:
print ('The top 5 states as per GDP Per Capita are:\n   ', 
      "\n    ".join (list(ppp_gsdp_df.head().State)))

print()

print ('The bottom 5 states as per GDP Per Capita are:\n   ', 
      "\n    ".join (list(ppp_gsdp_df.tail().State)))

print()

print ('The ratio of Top GDP Per Capita and Bottom GDP Per Capita is', 
       round(float(ppp_gsdp_df.head(1).Amount) / float(ppp_gsdp_df.tail(1).Amount), 2)  )

In [ ]:
for state in master_df['State'].unique():
    state_gdp  = master_df.loc[(master_df.State==state) & (master_df.Item=='Gross State Domestic Product'), 'Amount']
    
    primary_contrib = 100*float(master_df.loc[(master_df.State==state) & (master_df.Item=='PrimaryTotal'), 'Amount']) / float(state_gdp)
    secondary_contrib = 100*float(master_df.loc[(master_df.State==state) & (master_df.Item=='SecondaryTotal'), 'Amount']) / float(state_gdp)
    tertiary_contrib = 100*float(master_df.loc[(master_df.State==state) & (master_df.Item=='TertiaryTotal'), 'Amount']) / float(state_gdp)
    
    gdp_df = [state, 'PrimaryContribution', primary_contrib, 'C']
    master_df.loc[len(master_df)] = gdp_df
    gdp_df = [state, 'SecondaryContribution', secondary_contrib, 'C']    
    master_df.loc[len(master_df)] = gdp_df
    gdp_df = [state, 'TertiaryContribution', tertiary_contrib, 'C']    
    master_df.loc[len(master_df)] = gdp_df
    
master_df.reset_index(drop=True, inplace=True)
master_df['Amount'] = master_df['Amount'].astype(float)

In [ ]:
sns.catplot(y='State', x='Amount', hue='Item', data=master_df[master_df.Category=='C'], 
            kind = 'bar', height=12)
plt.title('Sector wise contribution to State GDP (%)', fontsize=14)
plt.show()

In [ ]:
df_ppp_gdp               = master_df[master_df.Item=='Per Capita GSDP (Rs.)'][['State', 'Amount']].reset_index(drop=True)
sz = df_ppp_gdp.Amount.size-1
df_ppp_gdp['percentile'] = df_ppp_gdp.rank()['Amount'].apply (lambda x : 100.0*(x-1)/sz)

In [ ]:
df_ppp_gdp['Item']       = 'Percentile'
df_ppp_gdp['Category']   = 'O'
df_ppp_gdp=df_ppp_gdp[['State', 'Item', 'percentile', 'Category']].rename(columns={'percentile' : 'Amount'})
master_df=pd.concat([master_df, df_ppp_gdp]).reset_index(drop=True)

In [ ]:
sns.set(style="white", color_codes=True)
plot = sns.jointplot(master_df[master_df.Item=='PrimaryContribution']['Amount'], 
              master_df[master_df.Item=='Percentile']['Amount'], kind='reg').\
                  set_axis_labels('Contribution to Primary Sector', 'PPP GDP Percentile')
plot.ax_marg_x.set_xlim(0, 100)
plot.ax_marg_y.set_ylim(0, 100)
plot.fig.suptitle("Impact of Primary Sector's contribution to the GDP Percentile ", y=1)
plt.show()
#-------------------#

In [ ]:
def get_cat(percentile):
    if percentile <= 20:
        return 'C4'
    elif percentile <= 50:
        return 'C3'
    elif percentile <= 85:
        return 'C2'
    return 'C1'

In [ ]:
categorized_states           = master_df[master_df.Item=='Percentile'][['State', 'Amount']].rename(columns={'Amount':'Percentile'})
categorized_states['GDPCat'] = categorized_states['Percentile'].apply(get_cat)

In [ ]:
master_df=pd.merge(left=master_df, right=categorized_states[['State', 'GDPCat']], on='State')

In [ ]:
df_category = pd.DataFrame(columns=['GDPCat', 'Item', 'Amount', 'Percentage'])
for cat in ['C1', 'C2', 'C3', 'C4']:
    df = master_df[(master_df.GDPCat==cat) &  
                   ( (master_df.Category.isin(['P', 'S', 'T'])) | (master_df.Item=='Gross State Domestic Product'))]
    df_agg = df.groupby(['GDPCat', 'Item']).sum().reset_index().sort_values('Amount')
    category_gdp = float(df_agg[df_agg.Item=='Gross State Domestic Product'].Amount)
    target_val = category_gdp*0.79
    sum_items=0
    item_list    = list(df_agg.Item)[::-1][1:]
    select_items = []
    for item in item_list:
        item_gdp = float(df_agg[df_agg.Item == item].Amount)
        df_agg.loc[df_agg.Item==item, 'Percentage'] = 100 * item_gdp / category_gdp
        sum_items += item_gdp
        if sum_items >= target_val:
            continue
        select_items.append(item)
    df_category = pd.concat([df_category, df_agg])
    print ('For category', cat, 'states, these top', len(select_items), 
           'contribute to the 80% of the GDP:')
    print ('\n  '.join(select_items))
    print ('')

In [ ]:
df_category = df_category[df_category.Item!='Gross State Domestic Product']
df_category.reset_index(drop = True, inplace = True)

In [ ]:
sns.barplot(x='Percentage', y='GDPCat', hue='Item', data=df_category, palette = 'Blues_d')
plt.title('Sub Sector wise contribution to State Category GDP', fontsize=14)

####################################################################################################

In [46]:
df_dropout = pd.read_csv(path+'/rs_session243_au570_1.1.csv')
df_dropout = df_dropout[['Level of Education - State', 'Primary - 2014-2015',
                         'Upper Primary - 2014-2015', 'Secondary - 2014-2015']]
df_dropout.columns = ['State', 'Primary', 'Upper Primary', 'Secondary']
UT_list = ['Delhi', 'Puducherry', 'Chandigarh', 'Dadra & Nagar Haveli', 'A & N Islands', 'Daman & Diu', 'Lakshadweep']

In [ ]:
df_dropout = df_dropout[~(df_dropout.State.isin(UT_list))]

In [ ]:
df_dropout.loc[df_dropout.State == 'Odisha', 'State'] = 'Odhisa'
df_dropout.loc[df_dropout.State == 'Uttrakhand', 'State'] = 'Uttarakhand'
df_dropout.loc[df_dropout.State == 'Chhatisgarh', 'State'] = 'Chattisgarh'

In [ ]:
df_drop_ppp=pd.merge(left=master_df[master_df.Item=='Per Capita GSDP (Rs.)'][['State', 'Amount']].rename(columns={'Amount' : 'PC_GSDP'}), 
         right=df_dropout, on='State')

In [ ]:
for dropout in ['Primary', 'Upper Primary', 'Secondary']:
    plot = sns.jointplot('PC_GSDP', dropout, df_drop_ppp, kind='reg')
    plot.ax_marg_x.set_xlim(0, None)
    plot.ax_marg_y.set_ylim(0, None)
    plot.set_axis_labels('Per Capita GDP of State', 'Dropout in '+dropout, fontsize=12)
    plt.subplots_adjust(top=0.9)
    plot.fig.suptitle('Checking the relation of '+dropout+' Dropout against Per Capita GDP', fontsize=14)
    plt.show()

In [ ]:
df_drop_ppp=pd.merge(left=master_df[master_df.Item=='Per Capita GSDP (Rs.)'][['State', 'Amount']].rename(columns={'Amount' : 'PC_GSDP'}), 
         right=df_dropout, on='State')

In [ ]:
for sector in ['PrimaryContribution', 'SecondaryContribution', 'TertiaryContribution', "Population ('00)"]:
    for level in ['Primary', 'Upper Primary', 'Secondary']:
        df_corr = pd.merge(left = master_df[master_df.Item==sector][['State', 'Amount']].rename(columns={'Amount':sector}), 
                           right = df_dropout, on='State')
        plot = sns.jointplot(sector, level, df_corr, kind='reg')
        plot.ax_marg_x.set_xlim(0, None)
        plot.ax_marg_y.set_ylim(0, None)
        plot.fig.suptitle('Checking the relation of '+level+' Dropout against '+sector.replace('Contribution', ''),
                          fontsize=14)
        plot.set_axis_labels('Contribution By '+sector.replace('Contribution', ''), 
                             'Dropout in School ('+level+')', fontsize=12)
        plt.show()